# Data Preprocessing

* Data cleaning: handle missing values, duplicates, inconsistant or invalid vallues, outliers

* Data reduction: reduce number of attributes, reduce number of attribute values

* Data transformation: attribute construction, normalization

* Data discretization: encode to numerical attributes

## Setting up the Notebook

### Import Required Packages

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

---

## Loading the Data

In [57]:
# Load file into pandas dataframe
df = pd.read_csv('../data/train.csv')
# df = pd.read_csv('../data/test.csv')

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points, each with 30 attributes.


---

## Data Cleaning

Before data cleaning, remove the known attributes that are not meaningful to our prediction model:
  * Meaningless idendifier: listing_id 
  * Attributes in free text: title, description, features, accessories
  * Attribute with the same value: eco_category, indicative_price
  * Attribute unlikely to affect price: curb_weight

In [58]:
columns_to_drop = [
    'listing_id',          # Meaningless identifier
    'title',               # Attributes in free text
    'description',
    'features',
    'accessories',
    'eco_category',        # Attribute with the same value
    'indicative_price',
    'curb_weight',         # Attribute unlikely to affect price

    'original_reg_date',
    'lifespan',

    # 'make',
    # 'model',
    # 'type_of_vehicle',
    'transmission',
    'fuel_type',
    # 'no_of_owners',
    'opc_scheme',
    'lifespan',

    'category',
]

df = df.drop(columns=columns_to_drop)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 25000 data points, each with 14 attributes.


### Handle Missing Values
Firstly, for each of the columns with missing value, check the number of rows with NaN values.
There are 3 scenarios:
1. NaN values are valid, no need to remove.
2. NaN values are not valid, data points are big (e.g. fuel_type has 19121 rows with NaN values), set a default value.
3. NaN values are not valid, data points are small. Remove the data points with attribute with NaN values that are known to be invalid. 

In [59]:
columns_to_check = [
    'make',
    'fuel_type',
    'manufactured',
    'power',
    'engine_cap',
    'mileage',
    'no_of_owners',
    'depreciation',
    'road_tax',
    'dereg_value',
    'omv',
    'arf'
]

# Calculate the number of NaN values in each specified column
# nan_counts = df[columns_to_check].isna().sum()
nan_counts = df.isna().sum()

# Print the number of NaN values for each column
for column, count in nan_counts.items():
    print(f"Column '{column}' has {count} rows with NaN values.")

Column 'model' has 0 rows with NaN values.
Column 'manufactured' has 7 rows with NaN values.
Column 'reg_date' has 0 rows with NaN values.
Column 'type_of_vehicle' has 0 rows with NaN values.
Column 'power' has 2640 rows with NaN values.
Column 'engine_cap' has 596 rows with NaN values.
Column 'depreciation' has 507 rows with NaN values.
Column 'coe' has 0 rows with NaN values.
Column 'road_tax' has 2632 rows with NaN values.
Column 'dereg_value' has 220 rows with NaN values.
Column 'mileage' has 5304 rows with NaN values.
Column 'omv' has 64 rows with NaN values.
Column 'arf' has 174 rows with NaN values.
Column 'price' has 0 rows with NaN values.


In [6]:
# Replace NaN values of attribute fuel_type with 'petrol'
# df['fuel_type'] = df['fuel_type'].fillna('petrol')

### Remove Exact Duplicates

In [60]:
df = df.drop_duplicates()

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 24987 data points, each with 14 attributes.


### (SKIP) Transform categorical value to numerical values

In [8]:
# categorical_columns = [
#     'make',
#     'model',
#     'type_of_vehicle',
#     'transmission',
#     'fuel_type',
#     'opc_scheme',
# ]

# le = LabelEncoder()
# for column in categorical_columns:
#   df[column] = le.fit_transform(df[column])

### Transform date time attributes to numerical values

In [61]:
df['reg_date'] = pd.to_datetime(df['reg_date'], format='%d-%b-%Y')
df['reg_year'] = df['reg_date'].dt.year
df = df.drop(columns=['reg_date'])

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 24987 data points, each with 14 attributes.


### Handle power

In [62]:
#################################### power ####################################
# Step: fill in the missing values in column 'power'
# The power of cars for a certain model are very likely to be similar
# So we take the average values of power of each car model
# If there are still missing values, we take the average of 'type_of_vehicle'
mean_values = df.groupby('model')['power'].transform('mean')
df.loc[:, 'power'] = df['power'].fillna(mean_values)
df.loc[:, 'power'] = df['power'].round()

mean_values = df.groupby('type_of_vehicle')['power'].transform('mean')
df.loc[:, 'power'] = df['power'].fillna(mean_values)
df.loc[:, 'power'] = df['power'].round()

### Handle engine_cap

In [63]:
#################################### engine_cap ####################################
# Step: fill in the missing values in column 'engine_cap'
# We do the same as we did in step 3 here
mean_values = df.groupby('model')['engine_cap'].transform('mean')
df.loc[:, 'engine_cap'] = df['engine_cap'].fillna(mean_values)
df.loc[:, 'engine_cap'] = df['engine_cap'].round()

mean_values = df.groupby('type_of_vehicle')['engine_cap'].transform('mean')
df.loc[:, 'engine_cap'] = df['engine_cap'].fillna(mean_values)
df.loc[:, 'engine_cap'] = df['engine_cap'].round()

### Handle road_tax

In [64]:
from sklearn.linear_model import LinearRegression

#################################### road_tax ####################################
# Step: we handle road_tax NaN values, around 2,600 in total.
# First, we fill up NaN values w.r.t. engine_cap
road_tax_dict = {}
for index, row in df.iterrows():
    if row['engine_cap'] not in road_tax_dict.keys() or pd.isna(road_tax_dict[row['engine_cap']]):
        road_tax_dict[row['engine_cap']] = row['road_tax']

for index, row in df.iterrows():
    if pd.isna(row['road_tax']):
        df.loc[index, 'road_tax'] = road_tax_dict[row['engine_cap']]

# After filling up with road_tax w.r.t. engine_cap, there are still around 1,000 NaN values
# We use linear approximation to fill up these values after using EDA
df_tmp = df.dropna(subset=['road_tax'])

x = df_tmp['engine_cap'].values.reshape(-1, 1)
y = df_tmp['road_tax'].values
model = LinearRegression()
model.fit(x, y)

# Find indices with missing road_tax values and predict
missing_indices = df[df['road_tax'].isnull()].index
x_missing = df.loc[missing_indices, 'engine_cap'].values.reshape(-1, 1)
y_pred = model.predict(x_missing)
df.loc[missing_indices, 'road_tax'] = [round(yi) for yi in y_pred]

### Handle manufactured

In [65]:

#################################### manufactured ####################################
df['manufactured'].fillna(df['reg_year'], inplace=True)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 24987 data points, each with 14 attributes.


C:\Users\hanli\AppData\Local\Temp\ipykernel_41692\461829635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['manufactured'].fillna(df['reg_year'], inplace=True)


### Handle mileage

In [66]:
mean_values = df.groupby('reg_year')['mileage'].transform('mean')
df.loc[:, 'mileage'] = df['mileage'].fillna(mean_values)
df.loc[:, 'mileage'] = df['mileage'].round()

# remaining NaN values apply the avg singapore mileage per year (17500)
df['mileage'].fillna((2024 - df['reg_year']) * 17500, inplace=True)

C:\Users\hanli\AppData\Local\Temp\ipykernel_41692\1131364436.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['mileage'].fillna((2024 - df['reg_year']) * 17500, inplace=True)


### Handle the remaining NaN values

In [67]:
df = df.drop(columns=['model', 'type_of_vehicle'])

columns_with_invalid_NaN= [
    # 'make',
    # 'fuel_type',
    'manufactured',
    'power',
    'engine_cap',
    'depreciation',
    'road_tax',
    'dereg_value',
    'mileage',
    'omv',
    'arf'
]

df = df.dropna(subset=columns_with_invalid_NaN)

# df[columns_with_invalid_NaN] = df[columns_with_invalid_NaN].fillna(0)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

There are 24253 data points, each with 12 attributes.


### (SKIP) Handle category attribute

In [10]:
# Replace '-' with an empty string
df['category'] = df['category'].replace('-', '')

# Split the 'category' column into lists
df['category_list'] = df['category'].str.split(', ')

# Handle empty strings by replacing them with empty lists
df['category_list'] = df['category_list'].apply(lambda x: [] if x == [''] else x)

# Import itertools for flattening lists
from itertools import chain

# Flatten the list of lists to a single list
all_categories = list(chain.from_iterable(df['category_list']))

# Get the unique categories
unique_categories = set(all_categories)

# Print the number of unique categories
print(f"Number of unique categories: {len(unique_categories)}")
print("Unique categories:", unique_categories)

# Initialize the MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the category lists
category_dummies = mlb.fit_transform(df['category_list'])

# Create a DataFrame with the one-hot encoded categories
category_df = pd.DataFrame(category_dummies, columns=mlb.classes_, index=df.index)

# Concatenate the new dummy columns to the original DataFrame
df = pd.concat([df, category_df], axis=1)

# Drop the temporary 'category_list' column if desired
df.drop('category_list', axis=1, inplace=True)
df.drop('category', axis=1, inplace=True)

num_records, num_attributes = df.shape

print("There are {} data points, each with {} attributes.". format(num_records, num_attributes))

Number of unique categories: 14
Unique categories: {'rare & exotic', 'opc car', 'vintage cars', 'consignment car', 'sta evaluated car', 'parf car', 'direct owner sale', 'coe car', 'sgcarmart warranty cars', 'hybrid cars', 'premium ad car', 'low mileage car', 'imported used vehicle', 'almost new car'}
There are 17021 data points, each with 33 attributes.


---

## Saving the Data

In [68]:
file_name = 'train_preprocessed.csv'
# file_name = 'test_preprocessed.csv'


# Check if the file exists
if os.path.exists(file_name):
    # Delete the file
    os.remove(file_name)
    print(f"Existing file '{file_name}' has been deleted.")

# Save the DataFrame to CSV
df.to_csv(file_name, index=False)
print(f"DataFrame has been saved to '{file_name}'.")

Existing file 'train_preprocessed.csv' has been deleted.
DataFrame has been saved to 'train_preprocessed.csv'.


## Training the model

### Prepare training datasets

In [71]:
y = df['price']
X = df.drop(columns=['price'])

### Hyperparameter tunning on various models

In [70]:
import numpy as np

from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_validate

# Only considered hyperparameter: max depth of trees
param_choices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# Keep track of results for visualization
param_to_scores = {}

for param in param_choices:

    # Train regressor with the current parameter setting
    # regressor = DecisionTreeRegressor(max_depth=param)
    # regressor = RandomForestRegressor(max_depth=param)
    regressor = GradientBoostingRegressor(max_depth=param)
    
    # Perform 10-fold cross_validations
    scores = cross_validate(regressor, X, y, cv=10, scoring='neg_root_mean_squared_error', return_train_score=True)
    
    # Extract the 10 RSME scores (training scores and validation scores) for each run/fold
    # The (-1) is only needed since we get the negative root mean squared errors (it's a sklearn thing)
    rsme_train = scores['train_score'] * (-1)
    rsme_valid = scores['test_score'] * (-1)
    
    ## Keep track of all num_folds f1 scores for current param (for plotting)
    param_to_scores[param] = (rsme_train, rsme_valid)
    
    ## Print statement for some immediate feedback (values in parenthesis represent the Standard Deviation)
    print('param = {}, RSME training = {:.1f} ({:.1f}), RSME validation = {:.1f} ({:.1f})'
          .format(param, np.mean(rsme_train), np.std(rsme_train), np.mean(rsme_valid), np.std(rsme_valid)))

param = 1, RSME training = 35716.3 (349.7), RSME validation = 40153.0 (3744.9)
param = 2, RSME training = 23833.4 (377.2), RSME validation = 29767.0 (3211.1)
param = 3, RSME training = 15775.1 (113.0), RSME validation = 23532.3 (3962.7)
param = 4, RSME training = 10344.0 (119.2), RSME validation = 19963.1 (4482.7)
param = 5, RSME training = 7284.3 (123.9), RSME validation = 19052.9 (5527.8)
param = 6, RSME training = 5218.9 (46.1), RSME validation = 18923.0 (6369.6)
param = 7, RSME training = 3809.8 (43.2), RSME validation = 19560.6 (6609.8)
param = 8, RSME training = 2808.2 (35.5), RSME validation = 20814.7 (6361.6)


KeyboardInterrupt: 